In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [ ]:
parent_dir = os.path.split(os.getcwd())[0]

In [ ]:
zones = gpd.read_file(parent_dir + '\\Data\\New\\lms_zone_du_new.shp')
ovin_tb = pd.read_csv(parent_dir + '\\Data\\New\\lms_zone_ovin_travel_behaviour.csv', index_col=0)
lms_tb = pd.read_csv(parent_dir + '\\Data\\New\\lms_zone_lms_modal_split.csv', index_col=0)
lms_tt_td = pd.read_csv(parent_dir + '\\Data\\New\\lms_zone_lms_tt_td.csv')

In [ ]:
ovin = pd.read_csv(parent_dir + '\\Data\\New\\Ovin_final.csv', index_col=0)

In [ ]:
lms_orig = pd.read_csv(parent_dir + '\\Data\\New\\lms_modal_split_orig_abs.csv', index_col=0)
lms_dest = pd.read_csv(parent_dir + '\\Data\\New\\lms_modal_split_dest_abs.csv', index_col=0)

## Get average tt and td for each zone

In [ ]:
lms_tb['Bus_o'] = lms_tb.iloc[:, 4:6].sum(axis=1)

In [ ]:
lms_tb = lms_tb.drop(columns='Tram/Metro_o')

In [ ]:
distance = np.zeros(1406)
time = np.zeros(1406)

for i in range(6):
    dist = np.array(lms_tb.iloc[:, i + 1] * lms_tt_td.iloc[:, i + 1])
    dist[dist != dist] = 0

    distance += dist

    t = np.array(lms_tb.iloc[:, i + 1] * lms_tt_td.iloc[:, i + 7])
    t[t != t] = 0
    time += t

distance = distance / 100
time = time / 100

In [ ]:
time

## Get average for each DU

In [ ]:
tot_trips = lms_orig.sum(axis=1)

In [ ]:
du_time_lms = np.zeros(6)
du_dist_lms = np.zeros(6)

du_time_ovin = np.zeros(6)
du_dist_ovin = np.zeros(6)

for du in range(1, 7):
    idx = zones[zones.deg_urba == du].index

    # LMS
    trips = tot_trips[idx]
    t = time[idx]
    dist = distance[idx]

    du_time_lms[du - 1] = np.sum(trips * t) / np.sum(trips)
    du_dist_lms[du - 1] = np.sum(trips * dist) / np.sum(trips)

    # OViN
    trips_ovin = ovin_tb.Factor_o[idx]
    t_ovin = ovin_tb.Travel_time_o[idx]
    dist_ovin = ovin_tb.Distance_o[idx]

    du_time_ovin[du - 1] = np.sum(trips_ovin * t_ovin) / np.sum(trips_ovin)
    du_dist_ovin[du - 1] = np.sum(trips_ovin * dist_ovin) / np.sum(trips_ovin)


    

In [ ]:
np.sum(trips_ovin * t_ovin) /  np.sum(trips_ovin)

In [ ]:
du_dist_ovin

## Plot

In [ ]:
f, ax = plt.subplots(1, 2)
f.set_figwidth(10)

x = np.arange(1, 7)

ax[0].bar(x - 0.2, du_dist_ovin, width=0.35, color='firebrick', label='OViN')
ax[0].bar(x + 0.2, du_dist_lms, width=0.35, color='forestgreen', label='LMS')
ax[0].set_xticks(x)
ax[0].set_ylabel('Distance [km]')
ax[0].legend()
ax[0].set_title('Average travel distance OViN and LMS\n by degree of urbanisation')
ax[0].set_xlabel('Degree of urbanisation')
ax[0].set_axisbelow(True)
ax[0].grid(axis='y')

ax[1].bar(x - 0.2, du_time_ovin, width=0.35, color='firebrick', label='OViN')
ax[1].bar(x + 0.2, du_time_lms, width=0.35, color='forestgreen', label='LMS')
ax[1].set_xticks(x)
ax[1].set_ylabel('Travel time [min]')
ax[1].legend()
ax[1].set_title('Average travel time OViN and LMS\n by degree of urbanisation')
ax[1].set_xlabel('Degree of urbanisation')
ax[1].set_axisbelow(True)
ax[1].grid(axis='y')
